<a href="https://colab.research.google.com/github/jackie-Gung/Colab_ESAA/blob/main/2022_10_14_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Chapter 8. 텍스트 분석**
---

### **04. 텍스트 분류 실습 - 20 뉴스그룹 분류**
---

- **개요**
  - 텍스트 피처 벡터화
  - 희소 행렬 형태
  - 알고리즘: 로지스틱 회귀, 선형 서포트 벡터 머신,  나이브 베이즈

#### **1. 텍스트 정규화**

In [1]:
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all',random_state=156)

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd
print('target 클래스의 값과 분포도\n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들\n', news_data.target_names)

target 클래스의 값과 분포도
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
# 개별 데이터가 텍스트로 어떻게 구성되어 있는지 확인하기
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- **분석**
      1. 뉴스그룹 기사의 내용과 제목, 작성자, 소속, 이메일 등 다양한 전보를 가짐
      2. 내용을 제외한 다른 정보는 제거하기
         -> 제목, 소속 등 헤더와 푸터 정보들은 target 클래스 값과 유사한 경우가 많기 때문
         -> 이러한 정보를 포함하게 되면 예측 성능이 좋기 때문에 텍스트 분석의 의도를 벗어남

In [12]:
from sklearn.datasets import fetch_20newsgroups

In [13]:
# 헤더와 푸터 정보 제거하기
## subset ='train'으로 학습용 데이터만 추출
## remove = ('headers','footers','quotes') 내용만 추출

train_news = fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target

## remove = ('headers','footers','quotes') 내용만 추출

test_news = fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'), random_state=156)
X_test = test_news.data
y_test = test_news.target

In [14]:
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


#### **2. 피처 벡터화 변환 및 머신러닝 모델 학습/예측/평가**

##### **1) Count 기반**

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
# Count Vectorization으로 피처 벡터화 변환 수행하기
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

In [18]:
# 학습 데이터로 fit된 c.v를 이용해 테스트 데이터를 피처 벡처화 변환 수행하기
X_test_cnt_vect = cnt_vect.transform(X_test)
print('학습 데이터 텍스트의 CountVectorizer Shape: ', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape:  (11314, 101631)


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [20]:
# 로지스틱 회귀 적용하여 예측하기
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized logistic Regression 예측 정확도: {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized logistic Regression 예측 정확도: 0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


##### **2) TF-IDF 기반**

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF L.R 예측 정화도: {0:.3f}'.format(accuracy_score(y_test,pred)))

TF-IDF L.R 예측 정화도: 0.674


- **텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 방법**
  - 최적의 ML 알고리즘 선택
  - 최상의 피처 전처리 수행

In [23]:
# stop words 필터링 추가 / ngram 기본에서 (1,2)로 변경 / 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

In [24]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized L.R 예측 정확도: {0:.3f}'.format(accuracy_score(y_test,pred)))

TF-IDF Vectorized L.R 예측 정확도: 0.692


##### **3) GridSearchCV**

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'C': [0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('L.R. best C parameter: ',grid_cv_lr.best_params)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


In [ ]:
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized L.R 예측 정확도: {0:.3f}'.format(accuracy_score(y_test,pred)))

#### **3. 사이킷런 파이프라인 사용 및 GridSearchCV와의 결합**

##### **1) pipeline 기본**

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),
                     'lr_clf',LogisticRegression(C=10)])

In [ ]:
pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)
print(accuracy_score(y_test,pred))

##### **2) pipeline+GridSearchCv**

In [ ]:
pipeline = Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),
                     'lr_clf',LogisticRegression()])

params = {'tfidf_vect_ngram_range':[(1,1),(1,2),(1,3)],
          'tfidf_vect_max_df': [100,300,700],
          'lr_clf_C':[1,5,10]}

grid_cv_pipe = GridSearchCV(pipeline,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train,y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)
pred = grid_cv_pipe.predict(X_test)
print(accuracy_score(y_test,pred))